# Configuração final da estrutura de diretórios

* data_ex (catalog) 
    * desafio (schema)
        * lhdw (volume)
        * bronze (volume)
            * bronze_dim_categoria_produto
            * bronze_dim_cliente 
            * bronze_dim_data 
            * bronze_dim_localidade
            * bronze_dim_produto 
            * bronze_fato_vendas  
        * silver (volume)
            * silver_dim_categoria_produto
            * silver_dim_cliente 
            * silver_dim_data 
            * silver_dim_localidade
            * silver_dim_produto 
            * silver_fato_vendas
        * gold  (volume)
            * dim_categoria_produto
            * dim_cliente 
            * dim_data 
            * dim_localidade
            * dim_produto 
            * fato_vendas

Tornar o escrito a cima em um diagrama para ser um dos anexos ao storytelling

In [0]:
# Criação do catalog
spark.sql("create catalog if not exists data_ex")

# Criação do schema
spark.sql("create schema if not exists data_ex.desafio")

# Criação dos volumes
spark.sql("create volume if not exists data_ex.desafio.lhdw")
spark.sql("create volume if not exists data_ex.desafio.bronze")
spark.sql("create volume if not exists data_ex.desafio.silver")
spark.sql("create volume if not exists data_ex.desafio.gold")

# # Criação dos diretórios
# paths = [
#     "/Volumes/data_ex/desafio/lhdw/processar"
# ]

# for path in paths:
#     # Tenta ler os caminhos já existentes
#     try:
#         dbutils.fs.ls(path)
#     # Se o caminho não existir, uma excecao é lancada e um novo caminho é criado
#     except:
#         dbutils.fs.mkdirs(path)

In [0]:
dbutils.fs.rm("/Volumes/data_ex/desafio/lhdw/processar", True)
spark.sql("drop volume if exists data_ex.desafio.lhdw")
spark.sql("drop volume if exists data_ex.desafio.bronze")
spark.sql("drop volume if exists data_ex.desafio.silver")
spark.sql("drop volume if exists data_ex.desafio.gold")


# Importação pelo Github

### Função de download

In [0]:
import urllib.request

# Função para baixar os arquivos
def download_dataset(csv_files, url, lhdw_path):
    # Tenta realizar o download dos arquivos
    try :
        for file in csv_files:
            # url+file corresponde a url de download
            # lhdw_path+file corresponde ao caminho onde o arquivo será salvo(lhdw_path) e, nome o formato do arquivo(file)
            urllib.request.urlretrieve(url+file, lhdw_path+file)
            print(f"Csv {file} baixado com sucesso!")
    # Caso não consigua realizar o download, uma exceção é lançada
    except Exception as e:
        print(f"Erro ao baixar o arquivo: {file} - {e}")

### Utilizaçao da função de download

In [0]:
# Lista de .csv
csv_files = [
    "categoria_produto.csv",
    "cliente.csv",
    "data.csv",
    "localidade.csv",
    "produto.csv",
    "vendas_part1.csv",
    "vendas_part2.csv",
    "vendas_part3.csv",
    "vendas_part4.csv"
]

# Reposotório do dataset para o desafio
url = "https://github.com/andrerosa1977/dataexperts2026/blob/main/" 
lhdw_path = "/Volumes/data_ex/desafio/lhdw/"

download_dataset(csv_files, url, lhdw_path)


In [0]:
import os

os.listdir("/Volumes/data_ex/desafio/lhdw")

# Bronze Layer

## Configuração do servidor para trabalho no Bronze Layer

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Iniciar a SparkSession com configurações otimizadas
spark = SparkSession.builder \
    .appName("Load Data Bronze") \
    .config("spark.sql.shuffle.partitions", "200")  \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Define um número fixo de partições para shuffle, melhorando o paralelismo                 
# Define o tamanho máximo de partições para evitar muitos arquivos pequenos        
# Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita    
# Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados


## Trabalhos no Bronze Layer

### Definindo o Log de carga
* id_carga (string (uuid))
* id_job (string (uuid))
* nome_arquivo (string)
* fonte (string)
* camada (string)
* path_destino (string)
* data_inicio (utc_timestamp)
* data_fim (utc_timestamp)
* duracao_ms (bigint)
* registros_lidos (bigint)
* registros_gravados (bigint)
* status (string)
* mensagem_erro (string)
* data_execucao (date(yyyy-MM-dd))

In [0]:
# Importação das bibliotecas/funções necessárias
# from pyspark.sql.functions import *
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, TimestampType, LongType, DateType, TimestampType, BooleanType, DoubleType

# Definição dos diretorios de trabalho
lhdw_path = "/Volumes/data_ex/desafio/lhdw"
bronze_path = "/Volumes/data_ex/desafio/bronze"

# Arquivos csv a serem trabalhados
csv_files = [
    "categoria_produto.csv",
    "cliente.csv",
    "data.csv",
    "localidade.csv",
    "produto.csv",
    "vendas_part1.csv",
    "vendas_part2.csv",
    "vendas_part3.csv",
    "vendas_part4.csv"
]

# Lista dos nomes dos schemas em ordem de execução
schemas = [
    "schema_categoria",
    "schema_cliente",
    "schema_data",
    "schema_localidade",
    "schema_produto",
    "schema_vendas"
]

# Definição manual do schemas a serem aplicados nos arquivos csv (futuros parquets)
schema_categoria = StructType([
    StructField("categoria_id", LongType(), True),
    StructField("categoria_nome", StringType(), True),
])

schema_cliente = StructType([
    StructField("cliente_id", LongType(), True),
    StructField("nome_cliente", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("cidade", StringType(), True)
])

schema_data = StructType([
    StructField("data_id", LongType(), True),
    StructField("data", DateType(), True),
    StructField("ano", IntegerType(), True),
    StructField("mes", IntegerType(), True),
    StructField("dia", IntegerType(), True),
    StructField("dia_semana", StringType(), True), 
    StructField("final_de_semana", BooleanType(), True)
])

schema_localidade = StructType([
    StructField("localidade_id", LongType(), True),
    StructField("estado", StringType(), True),
    StructField("cidade", StringType(), True),
    ])  

schema_produto = StructType([
    StructField("produto_id", LongType(), True),
    StructField("preco_lista", DoubleType(), True),
    StructField("categoria_nome", StringType(), True)
])

schema_vendas = StructType([
    StructField("venda_id", LongType(), True),
    StructField("cliente_id", LongType(), True),
    StructField("produto_id", LongType(), True),
    StructField("data_id", LongType(), True),
    StructField("categoria_id", LongType(), True),
    StructField("localidade_id", LongType(), True),
    StructField("quantidade", LongType(), True),
    StructField("preco_lista", DoubleType(), True),
    StructField("valor_total", DoubleType(), True)
])


        

In [0]:


# Criando uma lista para armazenar os Sparks DataFrames
spark_df = []

# Criação do Spark DataFrame, leitura dos csv, aplicação dos schemas e escrita em parquet 
def bronze_work(csv_file, schemas):
    for file, schema in zip(csv_files, schemas):
        # Tenta ler os arquivos csv em lhdw_path
        try:
            # Se schema não for vendas, aplica o schema correspondente a file
            if schema is not "schema_vendas":
                df = spark.read.csv(f"{lhdw_path}/{file}", header=True, schema=schema)
                print(f"Leitura do arquivo {file} bem suceddida.")
            
            # Se for vendas, deve repetir a leirura do arquivo n vezes
            else:
                # Tenta ler o caminho no qual vendas esta
                try:
                    paths = dbutils.fs.ls(lhdw_path)
                    # Conta o numero csv vendas existentes
                    n = len([n for n in paths if n.name.startswith("vendas")])
                    # Aplica um loop para ler todos os arquivos vendas
                    for i in range(1, n+1):
                        try:
                            df = spark.read.csv(f"{lhdw_path}/vendas_part{i}.csv", header=True, schema=schema)
                            print(f"Leitura do arquivo vendas_part{i}.csv bem suceddida.")
                        except Exception as e:
                            print(f"Erro ao ler o arquivo {lhdw_path}/vendas_part{i}.csv: {e}")
                except Exception as e:
                    print("ERRO")
                    print(f"Erro ao ler o caminho no qual no qual vendas esta {e}")
            
        except Exception as e:
            print(f"Erro ao ler o arquivo {file}: {e}")

bronze_work(csv_files, schemas)

## Salvar no Bronze Layer

## Limpar memória